# Map session dates to raw file paths

In [45]:
import os
import glob
import pandas as pd
import src.paths

dates = [
    '2022-07-27', '2022-07-28', '2022-08-01',
    '2022-08-03', '2022-08-04', '2022-08-06',
    '2022-08-08', '2022-08-10', '2022-08-11',
    '2022-08-12', '2022-08-13', '2022-08-15',
    '2022-09-12', '2022-09-14', '2022-09-16',
]

In [51]:
def session_path(date):
    datadir = src.paths.DATA / date
    if not os.path.isdir(datadir):
        raise ValueError(f"data directory {datadir} not found")
    
    pattern = datadir / f'*.imec1.lf.meta'
    match = glob.glob(str(datadir / pattern))[0]
    file = os.path.split(match)[-1]
    return file.replace(f'.imec1.lf.meta', '')

In [53]:
for date in dates:
    print(session_path(date))

2022-07-27-07-41-00_M015_SAL_PFC_HPC_0_0_0mpk_g0_t0
2022-07-28-13-19-00_M016_SAL_PFC_HPC_0_0_0mpk_g0_t0
2022-08-01-04-30-00_M015_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-08-03-12-45-00_M015_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-08-04-03-45-00_M016_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-08-06-05-20-00_M016_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0
2022-08-10-03-33-00_M017_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-08-11-01-55-00_M018_SAL_mPFC_HPC_0_0_0mpk_g0_t0
2022-08-12-02-35-00_M017_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-08-13-03-57-00_M018_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-08-15-02-25-00_M018_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-09-12-04-40-00_M023_SAL_mPFC_HPC_0_0_0mpk_g0_t0
2022-09-14-03-35-00_M023_RSK_mPFC_HPC_3_10_30mpk_g0_t0
2022-09-16-02-10-00_M023_RSK_mPFC_HPC_3_10_30mpk_g0_t0


# Map from session dates to recording numbers

In [55]:
if os.name == 'posix':
    import pathlib
    pathlib.WindowsPath = pathlib.PosixPath

import json

In [59]:
blob = pd.read_pickle(src.paths.REF / 'HPC_lfp_paths.pkl')
out = {}
for i, path in enumerate(blob):
    file = os.path.split(path)[-1]
    date = file[:10]
    if date.endswith('2022'):
        date = f'2022-{date[:5]}'
    out[date] = i

In [61]:
with open(src.paths.REF / 'date_recid.json', 'w') as f:
    json.dump(out, f)

# Map from session date to LFP start offset

In [98]:
df = pd.read_csv(src.paths.REF / 'recid_CA1_DG_id_modified.csv')
out = {}
for i, row in df.iterrows():
    date = row['Name_file'][:10]
    start = row['Lfp start']
    end = row['Lfp end']
    out[date] = [start, end]

In [100]:
with open(src.paths.REF / 'date_recoffset.json', 'w') as f:
    json.dump(out, f)

# Load behavioral data using recording number

In [66]:
import numpy as np

In [96]:
def load_behavior(recnum):
    datadir = src.paths.DATA / 'behavior'
    x = np.load(datadir / 'x_aln.npy', allow_pickle=True)[recnum]
    y = np.load(datadir / 'y_aln.npy', allow_pickle=True)[recnum]
    yaw = np.load(datadir / 'yaw_aln.npy', allow_pickle=True)[recnum]
    speed = np.load(datadir / 'speed_aln.npy', allow_pickle=True)[recnum]
    t = np.arange(len(x)) / 100
    return pd.DataFrame({
        'x': x,
        'y': y,
        'yaw': yaw,
        'speed': speed,
    }, index=t)

In [97]:
load_behavior(0)

,x,y,yaw,speed
0.00,9.494136,-29.672452,33.877280,304.092654
0.01,11.315372,-35.419425,33.877692,378.882912
0.02,12.989973,-40.598675,33.890433,420.289312
0.03,14.529093,-45.292913,33.927162,438.826685
0.04,15.890402,-49.563284,33.933638,441.202489
...,...,...,...,...
7202.09,-96.021851,26.185924,-136.825385,1.135706
7202.10,-96.010455,26.201583,-136.839977,0.782772
7202.11,-96.039774,26.180039,-136.826041,0.842430
7202.12,-96.035179,26.172649,-136.817985,0.829421
